In [1]:
import pandas as pd
import numpy as np

df_energy = pd.read_csv("Usecase1_Dataset.csv")

In [2]:
len(df_energy)

11746

In [3]:
len(df_energy.dropna())/len(df_energy)
#as nulls are almost 80 perc of whole data points hence can't be dropped

0.7986548612293547

In [4]:
len(df_energy.dropna())

9381

In [5]:
#I am choosing only those data points for which energy star score is available as 
# assuming or filling them might change data distributions
df_energy = df_energy[df_energy["ENERGY STAR Score"] != "Not Available"].reset_index().drop(["index","Order"],axis=1)

In [6]:
df_energy = df_energy.replace("Not Available",np.nan)
df_energy.info()
#Workable data info is below

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9642 entries, 0 to 9641
Data columns (total 59 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Property Id                                                 9642 non-null   int64  
 1   Property Name                                               9642 non-null   object 
 2   Parent Property Id                                          9642 non-null   object 
 3   Parent Property Name                                        9642 non-null   object 
 4   BBL - 10 digits                                             9640 non-null   object 
 5   NYC Borough, Block and Lot (BBL) self-reported              9640 non-null   object 
 6   NYC Building Identification Number (BIN)                    9510 non-null   object 
 7   Address 1 (self-reported)                                   9642 non-null   object 
 8 

In [7]:
#Here i am finding out the percentage not null data percentage in desending order
missing_data_perc = dict()
for i in df_energy.columns:
    missing_data_perc[i] = len(df_energy[~df_energy[i].isnull()])/len(df_energy)
    
missing_data_perc = pd.Series(missing_data_perc) * 100
missing_data_perc.sort_values(ascending=False) 

Property Id                                                   100.000000
Property GFA - Self-Reported (ft²)                            100.000000
Year Built                                                    100.000000
ENERGY STAR Score                                             100.000000
Site EUI (kBtu/ft²)                                           100.000000
Largest Property Use Type - Gross Floor Area (ft²)            100.000000
Largest Property Use Type                                     100.000000
List of All Property Use Types at Property                    100.000000
Primary Property Type - Self Selected                         100.000000
Postal Code                                                   100.000000
Address 1 (self-reported)                                     100.000000
Number of Buildings - Self-reported                           100.000000
Occupancy                                                     100.000000
Release Date                                       

In [8]:
#identifying columns with less than 40% missing values or 60% not null values
missing_data_perc_filter_40_perc_missing = missing_data_perc[missing_data_perc.sort_values(ascending=False) > 60]



In [9]:
#List of columns which will be used ...rest will be dropped as they wont do any good either
list(missing_data_perc_filter_40_perc_missing.index)

['Property Id',
 'Property Name',
 'Parent Property Id',
 'Parent Property Name',
 'BBL - 10 digits',
 'NYC Borough, Block and Lot (BBL) self-reported',
 'NYC Building Identification Number (BIN)',
 'Address 1 (self-reported)',
 'Postal Code',
 'Street Number',
 'Street Name',
 'Borough',
 'DOF Gross Floor Area',
 'Primary Property Type - Self Selected',
 'List of All Property Use Types at Property',
 'Largest Property Use Type',
 'Largest Property Use Type - Gross Floor Area (ft²)',
 'Year Built',
 'Number of Buildings - Self-reported',
 'Occupancy',
 'Metered Areas (Energy)',
 'Metered Areas  (Water)',
 'ENERGY STAR Score',
 'Site EUI (kBtu/ft²)',
 'Weather Normalized Site EUI (kBtu/ft²)',
 'Weather Normalized Site Electricity Intensity (kWh/ft²)',
 'Weather Normalized Site Natural Gas Intensity (therms/ft²)',
 'Weather Normalized Source EUI (kBtu/ft²)',
 'Natural Gas Use (kBtu)',
 'Weather Normalized Site Natural Gas Use (therms)',
 'Electricity Use - Grid Purchase (kBtu)',
 'Weathe

In [10]:
#remove duplicate columns which are related to property loction as BBL-10 Digit is enough along with pincode to tell about the 
#locality
duplicate_location_columns = ['Property Name',
       'Parent Property Name','Street Number','Street Name',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Street Name', 'Borough','Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'NTA']

In [11]:
#Carrying only relavant columns having more than 40 perc not null values
df_energy = df_energy[list(missing_data_perc_filter_40_perc_missing.index)]

In [12]:
df_energy = df_energy.drop(duplicate_location_columns,axis=1) #removing duplicate location columns as stated before

In [13]:
# As per Data set, segregating  BBL code to 3 columns for NYC Borough, block and lot codes which along with
#pin code will give info about building and locality

df_energy["NYC Borough"] = df_energy["BBL - 10 digits"].apply(lambda x:str(x)[0:1])
df_energy["NYC Block"] = df_energy["BBL - 10 digits"].apply(lambda x:str(x)[1:6])
df_energy["NYC Lot"] = df_energy["BBL - 10 digits"].apply(lambda x:str(x)[6:10])

In [14]:
#dropping BBL code as it is no longer necessary
df_energy = df_energy.drop(['BBL - 10 digits','Release Date'],axis=1)

In [15]:
#Reorganising Dataset
df_energy = pd.concat([df_energy.drop('ENERGY STAR Score',axis=1),df_energy["ENERGY STAR Score"]],axis=1)

In [16]:
#Dataset info with columns to be used in regression 
df_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9642 entries, 0 to 9641
Data columns (total 35 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Property Id                                                 9642 non-null   int64  
 1   Parent Property Id                                          9642 non-null   object 
 2   Postal Code                                                 9642 non-null   object 
 3   DOF Gross Floor Area                                        9579 non-null   float64
 4   Primary Property Type - Self Selected                       9642 non-null   object 
 5   List of All Property Use Types at Property                  9642 non-null   object 
 6   Largest Property Use Type                                   9642 non-null   object 
 7   Largest Property Use Type - Gross Floor Area (ft²)          9642 non-null   object 
 8 

In [17]:
#Replacing standalone bulding parent property id with property id.
for i in range(len(df_energy)):
    if(df_energy["Parent Property Id"][i] == "Not Applicable: Standalone Property"):
        df_energy["Parent Property Id"][i] = df_energy["Property Id"][i]        

<ipython-input-17-e02479c7125b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy["Parent Property Id"][i] = df_energy["Property Id"][i]


In [19]:
#Organising Category columns under one list for ease
cat_columns = ['Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type','Metered Areas (Energy)','Water Required?','Metered Areas  (Water)',
       'DOF Benchmarking Submission Status']

In [20]:
#Transforming categories to numbers for regression
from sklearn.preprocessing import LabelEncoder
for col in cat_columns:
    try:
        df_energy[col] = LabelEncoder().fit_transform(df_energy[col])
    except:
        df_energy[col] = df_energy[col].fillna(value=" ")
        df_energy[col] = LabelEncoder().fit_transform(df_energy[col])                

In [21]:
df_energy_fliter = df_energy

# Data Cleaning 

In [22]:
#below code was used to find some junk values rows which were troubling.
#df_energy_fliter[df_energy_fliter.eq('\u200b').any(1)].T
df_energy_fliter = df_energy_fliter.drop([95,190,9630,9631],axis=0).reset_index().drop('index',axis=1)

/home/kekarot/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [23]:
#Cleaning Postal code as some junk or multiple values were there after 6 character in some rows
df_energy_fliter["Postal Code"] = df_energy_fliter["Postal Code"].apply(lambda x:str(x)[:5]) 

In [24]:
#replacing null values with zeroes for row for integer type columns for now
df_energy_fliter = df_energy_fliter.replace(np.nan,0)

In [25]:
df_energy_fliter = df_energy_fliter.astype(float)

# Replacing mean values by property use type in Water resource variables
# As missing values were around 40 perc

In [26]:
for i in range(len(df_energy_fliter)):
    if((df_energy_fliter['Water Intensity (All Water Sources) (gal/ft²)'][i] == 0)
       or (df_energy_fliter['Water Intensity (All Water Sources) (gal/ft²)'][i] == 0)):
        
        cat_no = df_energy_fliter["Largest Property Use Type"][i]
        
        df_energy_fliter['Water Intensity (All Water Sources) (gal/ft²)'][i] = df_energy_fliter[df_energy_fliter["Largest Property Use Type"] == cat_no]['Water Intensity (All Water Sources) (gal/ft²)'].mean() 
        
        df_energy_fliter['Water Use (All Water Sources) (kgal)'][i] = df_energy_fliter[df_energy_fliter["Largest Property Use Type"] == cat_no]['Water Use (All Water Sources) (kgal)'].mean() 
        
        

# Keeping a parallel dataset for non correlated variables for reference below

In [27]:
corr_matrix = df_energy_fliter.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
# Drop features 
df_energy_fliter_non_corr =df_energy_fliter.drop(df_energy_fliter[to_drop], axis=1)

# Dataset summary to help with model and variable selection

#Observations for below(statsmodels.api)

1) Adjusted R-Sq of 0.812 using OLS shows that data has linearity 

2) kurtosis is on higher side ~2.29 shows that up side variance in target variable is there, also distribution is skewed. 

3) Also, high corelation is there among variables

4) Summary of df_energy_fliter_non_corr(hasnt been shown here) is almost same with Adjusted R-Sq of 0.802.

Conclusion:

1) Since dataset has linearity, along with minor skweness and kurtosis. Its best to use tree based algorithm with squarederror or linear as loss function.

2) Also tree based algorithms such as Decision trees, XG boost(XGBRegressor), GradientBoostingRegressor & XtraTreeRegressor are good at taking care of auto-correlation among variables on its own. Hence, It will save my time which will go in implementing cholesky decomposition or lower or upper triangular matrix or choosing variables manually by below p-values.

3) I will be using XG boost(XGBRegressor) as it is easier to optimize,tune and takes less time in training and creating a model file compared to others.


In [28]:
import statsmodels.api as sm

model = sm.OLS(df_energy_fliter["ENERGY STAR Score"],df_energy_fliter.drop("ENERGY STAR Score",axis=1))
result=model.fit()
print(result.summary2())

                                      Results: Ordinary least squares
Model:                           OLS                        Adj. R-squared (uncentered):          0.812     
Dependent Variable:              ENERGY STAR Score          AIC:                                  92302.3797
Date:                            2020-07-26 21:44           BIC:                                  92546.2777
No. Observations:                9638                       Log-Likelihood:                       -46117.   
Df Model:                        34                         F-statistic:                          1227.     
Df Residuals:                    9604                       Prob (F-statistic):                   0.00      
R-squared (uncentered):          0.813                      Scale:                                841.78    
------------------------------------------------------------------------------------------------------------
                                                          

In [46]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV 


X_train, X_test, y_train, y_test = train_test_split(df_energy_fliter.drop(["ENERGY STAR Score"]
                                                                           ,axis=1),df_energy_fliter["ENERGY STAR Score"] 
                                                                           ,test_size = 0.2)

#X_train, X_test, y_train, y_test = train_test_split(df_energy_fliter_non_corr.drop(["ENERGY STAR Score"]
#                                                    ,axis=1),df_energy_fliter_non_corr["ENERGY STAR Score"], test_size = 0.3)


gradient_boosted = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                               colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
                               importance_type='gain', interaction_constraints='',
                               learning_rate=0.1, max_delta_step=0, max_depth=5,
                               min_child_weight=5, monotone_constraints='()',
                               n_estimators=200, n_jobs=4, nthread=4, num_parallel_tree=1,
                               random_state=27, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=27, subsample=0.8, tree_method='exact', validate_parameters=1,
                               verbosity=None)



gradient_boosted.fit(X_train, y_train)
predictions_train = gradient_boosted.predict(X_train)
predictions = gradient_boosted.predict(X_test)

mae_train = mean_absolute_error(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print('XGBoost Performance on the train set: MAE = %0.4f' % mae_train)
print('XGBoost Performance on the train set: MSE = %0.4f' % mse_train)

print('XGBoost Performance on the test set: MAE = %0.4f' % mae)
print('XGBoost Performance on the test set: MSE = %0.4f' % mse)

XGBoost Performance on the train set: MAE = 6.0859
XGBoost Performance on the train set: MSE = 73.8125
XGBoost Performance on the test set: MAE = 8.3444
XGBoost Performance on the test set: MSE = 136.4559


#Test and train observations:

1) Testing and training MAE is looing good. However, MSE is quite high in testing which suggests that there could be outliers or it could be due to fact that variance explained by the model is only 0.81(Adj R-sq).

2) From Y_test and train description below,it is clear that MSE is high due to limited variance explained by model ie 0.81(Adj Rsquare). As Std dev , mean , quantile 0.25, 0.50 and 0.75 are almost same for both.

3) Also, the regression score of by Grid Search below is also 0.8180465993093975 which came at the time of hyper-parameter tuning. 

4) Hence it can be concluded that by both OLS statistical model and Grid parameters that it's the optimum result which can be achieved from this dataset as per my understanding.

#Final Results: Test MAE is dabbling between 7.9 - 8.8 for testing dataset between 20-30% of total usable datapoints that are around 9.6k out of total 11.7K.

#Possible Enhancements:

a) Better results can be achieved by replacing more missing values by mean values grouped by categories such as    building_use_type and others i have used. 

b) I belive variance explained by model can be pushed upto 0.85 if more time is spent on dataset.

In [35]:
y_test.describe()

count    1928.000000
mean       59.821058
std        29.789011
min         1.000000
25%        37.000000
50%        65.500000
75%        85.000000
max       100.000000
Name: ENERGY STAR Score, dtype: float64

In [37]:
y_train.describe()

count    7710.000000
mean       59.879637
std        30.037716
min         1.000000
25%        37.000000
50%        65.000000
75%        85.000000
max       100.000000
Name: ENERGY STAR Score, dtype: float64

In [250]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2),
 'n_estimators':range(100,500,100)   
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1,
 gamma=0, subsample=0.8, colsample_bytree=0.8,
 nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1,n_jobs=4,iid=False, cv=5)

gsearch1.fit(df_energy_fliter.drop(["ENERGY STAR Score"],axis=1),df_energy_fliter["ENERGY STAR Score"])
gsearch1.best_params_, gsearch1.best_score_

/home/kekarot/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


({'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 200},
 0.8180465993093975)

gsearch1.best_estimator_:

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=4, nthread=4, num_parallel_tree=1,
             random_state=27, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=27, subsample=0.8, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [281]:
df_output = pd.concat([pd.Series(y_test).reset_index().drop('index',axis=1),pd.Series(predictions).reset_index().drop('index',axis=1)],axis=1)

In [282]:
df_output = df_output.rename({0:"Predicted_Score"},axis=1)

In [283]:
df_output

,ENERGY STAR Score,Predicted_Score
0,81.0,74.125252
1,55.0,58.889820
2,36.0,51.447529
3,68.0,94.514458
4,97.0,95.798058
...,...,...
959,57.0,61.550503
960,55.0,51.570591
961,50.0,37.056435
962,78.0,74.052139
